<div class="markdown-google-sans">

# <strong>事先準備</strong>
</div>

<div class="markdown-google-sans">

## <strong>安裝plotly</strong>
</div>
範例會使用plotly表格呈現,請先安裝plotly<br/>


In [ ]:
!pip install plotly

<div class="markdown-google-sans">

## <strong>載入plotly和設定渲染器</strong>
</div>
並且為了colab可以瀏覽,請透過以下指令設定

In [ ]:
#匯入顯示表格套件
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
import plotly.io as pio
pio.renderers.default = 'colab'  # 或嘗試 'notebook' 或 'iframe' 渲染器
from google.colab import output
output.enable_custom_widget_manager()
init_notebook_mode(connected=True)



<div class="markdown-google-sans">

## <strong>範例欄位設定</strong>
</div>
定義顯示欄位<br/>

In [ ]:
from typing import List
#匯入顯示資料套件
import pandas as pd
from IPython.display import display,clear_output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
from IPython.display import display, clear_output

head_match = dict(values=['成交時間', '成交價','成交量'])
head_bid = dict(values=['買價1','買量1' ,'買價2','買量2','買價3','買量3','買價4','買量4','買價5','買量5'
                       ,'買價6','買量6','買價7','買量7','買價8','買量8','買價9','買量9','買價10','買量10' ])
head_off = dict(values=['賣價1','賣量1' ,'賣價2','賣量2','賣價3','賣量3','賣價4','賣量4','賣價5','賣量5'
                       ,'賣價6','賣量6','賣價7','賣量7','賣價8','賣量8','賣價9','賣量9','賣價10','賣量10' ])
head_implied = dict(values=['隱含買價','隱含買量','隱含賣價','隱含賣量','最高價','最低價','開盤價','收盤價','目前結算價','最新結算價'])

# 創建初始表格
fig_match = go.FigureWidget(data=[go.Table( header=head_match, cells=dict(values=list(zip(*[['','','']])), align='left'),)])
fig_match.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
fig_bid = go.FigureWidget(data=[go.Table( header=head_bid, cells=dict(values=list(zip(*[['','','','','','','','','','','','','','','','','','','','']])), align='left'),)])
fig_bid.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
fig_off = go.FigureWidget(data=[go.Table( header=head_off, cells=dict(values=list(zip(*[['','','','','','','','','','','','','','','','','','','','']])), align='left'),)])
fig_off.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
fig_implied = go.FigureWidget(data=[go.Table( header=head_implied, cells=dict(values=list(zip(*[['','','','','','','','','','']])), align='left'),)])
fig_implied.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
col_match = {header: idx for idx, header in enumerate(head_match['values'])}
col_bid = {header: idx for idx, header in enumerate(head_bid['values'])}
col_off = {header: idx for idx, header in enumerate(head_off['values'])}
col_implied = {header: idx for idx, header in enumerate(head_implied['values'])}

# 初始K線圖表格
bardata = [['', '', '', '', '', '', '', '', '']]
bardata_dd = pd.DataFrame(bardata)
bardata_dd.columns = ['商品代號', '商品種類', '日期', '時間', '開', '高', '低', '收', '成交量']

<div class="markdown-google-sans">

## <strong>安裝unitrade</strong>
</div>

In [ ]:
!pip install unitrade

<div class="markdown-google-sans">

# <strong>初始化</strong>
</div>

In [ ]:
##API初始&登入
import unitrade
from unitrade.unitrade import *
from unitrade.quote.fdata import *
#初始API
api = Unitrade()

<div class="markdown-google-sans">

# <strong>登入api</strong>
</div>

In [ ]:
#錯誤事件處理
api.on_error = lambda error: print(f'api錯誤 {error}')
#系統登入
#loginresponse = api.login(
    #"url", "帳號", "密碼", '憑證檔名', '憑證密碼')#url,帳號,密碼,憑證檔名,憑證密碼
loginresponse = api.login("http://test167.pfctrade.com", "80009809769", "Aa123456", 'PSC_A226097195_20250310.pfx', '12345678')
#顯示登入結果
print(loginresponse)
print(f"登入旗標{api.login_status_flag}")
print(f"交易帳號{api.get_accounts()}")


<div class="markdown-google-sans">

# <strong>查詢行情</strong>
</div>


<div class="markdown-google-sans">

## <strong>成交價量</strong>
</div>
透過query_tick_data_trade方法實作,帶入商品代碼


In [ ]:
productid='CME|NQ|202412|F|' #商品代碼
tickDataTradeResponse=api.fquote.query_tick_data_trade(exchange= productid.split('|')[0], symbol= productid.split('|')[1], ym=productid.split('|')[2], cp= productid.split('|')[3], strike=productid.split('|')[4] )
if tickDataTradeResponse.ok == True:
  tickdatatrade=tickDataTradeResponse.data
  cur = list(fig_match.data[0].cells.values)
  cur[col_match['成交時間']] =tickdatatrade.time
  cur[col_match['成交價']] = tickdatatrade.lastprice
  cur[col_match['成交量'] ]= tickdatatrade.lastvolume
  with fig_match.batch_update():
    fig_match.data[0].cells.values = cur
  display(fig_match)
else:
  print('結果 '+tickDataTradeResponse.error)

<div class="markdown-google-sans">

## <strong>最佳買價量</strong>
</div>
透過query_tick_data_bid方法實作,帶入商品代碼


In [ ]:
tickDataBidResponse=api.fquote.query_tick_data_bid(exchange= productid.split('|')[0], symbol= productid.split('|')[1], ym=productid.split('|')[2], cp= productid.split('|')[3], strike=productid.split('|')[4] )
if tickDataBidResponse.ok == True:
    tickdatabid=tickDataBidResponse.data
    cur = list(fig_bid.data[0].cells.values)
    cur[col_bid['買價1']] =tickdatabid.BidDOM1Price
    cur[col_bid['買量1']] =tickdatabid.BidDOM1Volume
    cur[col_bid['買價2']] =tickdatabid.BidDOM2Price
    cur[col_bid['買量2']] =tickdatabid.BidDOM2Volume
    cur[col_bid['買價3']] =tickdatabid.BidDOM3Price
    cur[col_bid['買量3']] =tickdatabid.BidDOM3Volume
    cur[col_bid['買價4']] =tickdatabid.BidDOM4Price
    cur[col_bid['買量4']] =tickdatabid.BidDOM4Volume
    cur[col_bid['買價5']] =tickdatabid.BidDOM5Price
    cur[col_bid['買量5']] =tickdatabid.BidDOM5Volume
    cur[col_bid['買價6']] =tickdatabid.BidDOM6Price
    cur[col_bid['買量6']] =tickdatabid.BidDOM6Volume
    cur[col_bid['買價7']] =tickdatabid.BidDOM7Price
    cur[col_bid['買量7']] =tickdatabid.BidDOM7Volume
    cur[col_bid['買價8']] =tickdatabid.BidDOM8Price
    cur[col_bid['買量8']] =tickdatabid.BidDOM8Volume
    cur[col_bid['買價9']] =tickdatabid.BidDOM9Price
    cur[col_bid['買量9']] =tickdatabid.BidDOM9Volume
    cur[col_bid['買價10']] =tickdatabid.BidDOM10Price
    cur[col_bid['買量10']] =tickdatabid.BidDOM10Volume
    with fig_bid.batch_update():
        fig_bid.data[0].cells.values = cur
    display(fig_bid)
else:
    print('tickDataBidResponse '+tickDataBidResponse.error)

<div class="markdown-google-sans">

## <strong>最佳賣價量</strong>
</div>
透過query_tick_data_offer方法實作,帶入商品代碼


In [ ]:
tickDataoffResponse=api.fquote.query_tick_data_offer(exchange= productid.split('|')[0], symbol= productid.split('|')[1], ym=productid.split('|')[2], cp= productid.split('|')[3], strike=productid.split('|')[4] )
if tickDataoffResponse.ok == True:
    tickdataOffer=tickDataoffResponse.data
    cur = list(fig_off.data[0].cells.values)
    cur[col_off['賣價1']] =tickdataOffer.OfferDOM1Price
    cur[col_off['賣量1']] =tickdataOffer.OfferDOM1Volume
    cur[col_off['賣價2']] =tickdataOffer.OfferDOM2Price
    cur[col_off['賣量2']] =tickdataOffer.OfferDOM2Volume
    cur[col_off['賣價3']] =tickdataOffer.OfferDOM3Price
    cur[col_off['賣量3']] =tickdataOffer.OfferDOM3Volume
    cur[col_off['賣價4']] =tickdataOffer.OfferDOM4Price
    cur[col_off['賣量4']] =tickdataOffer.OfferDOM4Volume
    cur[col_off['賣價5']] =tickdataOffer.OfferDOM5Price
    cur[col_off['賣量5']] =tickdataOffer.OfferDOM5Volume
    cur[col_off['賣價6']] =tickdataOffer.OfferDOM6Price
    cur[col_off['賣量6']] =tickdataOffer.OfferDOM6Volume
    cur[col_off['賣價7']] =tickdataOffer.OfferDOM7Price
    cur[col_off['賣量7']] =tickdataOffer.OfferDOM7Volume
    cur[col_off['賣價8']] =tickdataOffer.OfferDOM8Price
    cur[col_off['賣量8']] =tickdataOffer.OfferDOM8Volume
    cur[col_off['賣價9']] =tickdataOffer.OfferDOM9Price
    cur[col_off['賣量9']] =tickdataOffer.OfferDOM9Volume
    cur[col_off['賣價10']] =tickdataOffer.OfferDOM10Price
    cur[col_off['賣量10']] =tickdataOffer.OfferDOM10Volume
    with fig_off.batch_update():
        fig_off.data[0].cells.values = cur
    display(fig_off)
else:
    print('結果 '+tickDataoffResponse.error)

<div class="markdown-google-sans">

## <strong>隱含買賣價</strong>
</div>
透過query_tick_data_implied方法實作,帶入商品代碼


In [ ]:
tickDataImpliedResponse=api.fquote.query_tick_data_implied(exchange= productid.split('|')[0], symbol= productid.split('|')[1], ym=productid.split('|')[2], cp= productid.split('|')[3], strike=productid.split('|')[4] )
if tickDataImpliedResponse.ok == True:
  tickdataimplied=tickDataImpliedResponse.data
  cur = list(fig_implied.data[0].cells.values)
  cur[col_implied['隱含買價']] = tickdataimplied.ImpliedBidPrice
  cur[col_implied['隱含買量']] = tickdataimplied.ImpliedBidVolume
  cur[col_implied['隱含賣價']] = tickdataimplied.ImpliedOfferPrice
  cur[col_implied['隱含賣量']] = tickdataimplied.ImpliedOfferVolume
  with fig_implied.batch_update():
    fig_implied.data[0].cells.values = cur
  display(fig_implied)
else:
  print('結果 '+tickDataImpliedResponse.error)

<div class="markdown-google-sans">

## <strong>開收盤價</strong>
</div>
透過query_tick_data_open_close方法實作,帶入商品代碼


In [ ]:
tickDataOpencloseResponse=api.fquote.query_tick_data_open_close(exchange= productid.split('|')[0], symbol= productid.split('|')[1], ym=productid.split('|')[2], cp= productid.split('|')[3], strike=productid.split('|')[4] )
if tickDataOpencloseResponse.ok == True:
  tickdataopenclose=tickDataOpencloseResponse.data
  cur = list(fig_implied.data[0].cells.values)
  cur[col_implied['開盤價']] = tickdataopenclose.Opening
  cur[col_implied['收盤價']] = tickdataopenclose.Closing
  with fig_implied.batch_update():
    fig_implied.data[0].cells.values = cur
  display(fig_implied)
else:
  print('結果 '+tickDataOpencloseResponse.error)

<div class="markdown-google-sans">

## <strong>最高最低價</strong>
</div>
透過query_tick_data_high_low方法實作,帶入商品代碼


In [ ]:
tickDataHighLowResponse=api.fquote.query_tick_data_high_low(exchange= productid.split('|')[0], symbol= productid.split('|')[1], ym=productid.split('|')[2], cp= productid.split('|')[3], strike=productid.split('|')[4] )
if tickDataHighLowResponse.ok==True:
  tickdatahighlow=tickDataHighLowResponse.data
  cur = list(fig_implied.data[0].cells.values)
  cur[col_implied['最高價']] = tickdatahighlow.High
  cur[col_implied['最低價']] = tickdatahighlow.Low
  with fig_implied.batch_update():
    fig_implied.data[0].cells.values = cur
  display(fig_implied)
else:
  print('結果 '+tickDataOpencloseResponse.error)

<div class="markdown-google-sans">

## <strong>結算價</strong>
</div>
透過query_tick_data_settle方法實作,帶入商品代碼


In [ ]:
tickDataSettleResponse=api.fquote.query_tick_data_settle(exchange= productid.split('|')[0], symbol= productid.split('|')[1], ym=productid.split('|')[2], cp= productid.split('|')[3], strike=productid.split('|')[4] )
if tickDataSettleResponse.ok==True:
  tickdatasettle=tickDataSettleResponse.data
  cur = list(fig_implied.data[0].cells.values)
  cur[col_implied['目前結算價']] = tickdatasettle.CurrStl
  cur[col_implied['最新結算價']] = tickdatasettle.NewStl
  with fig_implied.batch_update():
    fig_implied.data[0].cells.values = cur
  display(fig_implied)
else:
  print('結果 '+tickDataSettleResponse.error)

<div class="markdown-google-sans">

# <strong>接收即時行情</strong>
</div>


<div class="markdown-google-sans">

## <strong>*定義接收處理fucttion*</strong>
</div>

<div class="markdown-google-sans">

### <strong>*成交價量*</strong>
</div>

In [ ]:
def on_tick_data_trade(tickdatatrade: FTickDataTrade):
  cur = list(fig_match.data[0].cells.values)
  cur[col_match['成交時間']] =tickdatatrade.time
  cur[col_match['成交價']] = tickdatatrade.lastprice
  cur[col_match['成交量'] ]= tickdatatrade.lastvolume
  with fig_match.batch_update():
    fig_match.data[0].cells.values = cur

<div class="markdown-google-sans">

### <strong>*最佳買價*</strong>
</div>

In [ ]:
def on_tick_data_bid(tickdatabid: FTickDataBid):
    cur = list(fig_bid.data[0].cells.values)
    cur[col_bid['買價1']] =tickdatabid.BidDOM1Price
    cur[col_bid['買量1']] =tickdatabid.BidDOM1Volume
    cur[col_bid['買價2']] =tickdatabid.BidDOM2Price
    cur[col_bid['買量2']] =tickdatabid.BidDOM2Volume
    cur[col_bid['買價3']] =tickdatabid.BidDOM3Price
    cur[col_bid['買量3']] =tickdatabid.BidDOM3Volume
    cur[col_bid['買價4']] =tickdatabid.BidDOM4Price
    cur[col_bid['買量4']] =tickdatabid.BidDOM4Volume
    cur[col_bid['買價5']] =tickdatabid.BidDOM5Price
    cur[col_bid['買量5']] =tickdatabid.BidDOM5Volume
    cur[col_bid['買價6']] =tickdatabid.BidDOM6Price
    cur[col_bid['買量6']] =tickdatabid.BidDOM6Volume
    cur[col_bid['買價7']] =tickdatabid.BidDOM7Price
    cur[col_bid['買量7']] =tickdatabid.BidDOM7Volume
    cur[col_bid['買價8']] =tickdatabid.BidDOM8Price
    cur[col_bid['買量8']] =tickdatabid.BidDOM8Volume
    cur[col_bid['買價9']] =tickdatabid.BidDOM9Price
    cur[col_bid['買量9']] =tickdatabid.BidDOM9Volume
    cur[col_bid['買價10']] =tickdatabid.BidDOM10Price
    cur[col_bid['買量10']] =tickdatabid.BidDOM10Volume
    with fig_bid.batch_update():
        fig_bid.data[0].cells.values = cur

<div class="markdown-google-sans">

### <strong>*最佳賣價*</strong>
</div>

In [ ]:
def on_tick_data_offer(tickdataOffer: FTickDataOffer):
    cur = list(fig_off.data[0].cells.values)
    cur[col_off['賣價1']] =tickdataOffer.OfferDOM1Price
    cur[col_off['賣量1']] =tickdataOffer.OfferDOM1Volume
    cur[col_off['賣價2']] =tickdataOffer.OfferDOM2Price
    cur[col_off['賣量2']] =tickdataOffer.OfferDOM2Volume
    cur[col_off['賣價3']] =tickdataOffer.OfferDOM3Price
    cur[col_off['賣量3']] =tickdataOffer.OfferDOM3Volume
    cur[col_off['賣價4']] =tickdataOffer.OfferDOM4Price
    cur[col_off['賣量4']] =tickdataOffer.OfferDOM4Volume
    cur[col_off['賣價5']] =tickdataOffer.OfferDOM5Price
    cur[col_off['賣量5']] =tickdataOffer.OfferDOM5Volume
    cur[col_off['賣價6']] =tickdataOffer.OfferDOM6Price
    cur[col_off['賣量6']] =tickdataOffer.OfferDOM6Volume
    cur[col_off['賣價7']] =tickdataOffer.OfferDOM7Price
    cur[col_off['賣量7']] =tickdataOffer.OfferDOM7Volume
    cur[col_off['賣價8']] =tickdataOffer.OfferDOM8Price
    cur[col_off['賣量8']] =tickdataOffer.OfferDOM8Volume
    cur[col_off['賣價9']] =tickdataOffer.OfferDOM9Price
    cur[col_off['賣量9']] =tickdataOffer.OfferDOM9Volume
    cur[col_off['賣價10']] =tickdataOffer.OfferDOM10Price
    cur[col_off['賣量10']] =tickdataOffer.OfferDOM10Volume
    with fig_off.batch_update():
        fig_off.data[0].cells.values = cur

<div class="markdown-google-sans">

### <strong>*隱含買賣價*</strong>
</div>

In [ ]:
def on_tick_data_implied(ticksataimplied: FTickDataImplied):
  cur = list(fig_implied.data[0].cells.values)
  cur[col_implied['隱含買價']] = tickdataimplied.ImpliedBidPrice
  cur[col_implied['隱含買量']] = tickdataimplied.ImpliedBidVolume
  cur[col_implied['隱含賣價']] = tickdataimplied.ImpliedOfferPrice
  cur[col_implied['隱含賣量']] = tickdataimplied.ImpliedOfferVolume
  with fig_implied.batch_update():
    fig_implied.data[0].cells.values = cur

<div class="markdown-google-sans">

### <strong>*開收盤價*</strong>
</div>

In [ ]:
def on_tick_data_open_close(tickdataopenclose: FTickDataOpenclose):
  cur = list(fig_implied.data[0].cells.values)
  cur[col_implied['開盤價']] = tickdataopenclose.Opening
  cur[col_implied['收盤價']] = tickdataopenclose.Closing
  with fig_implied.batch_update():
    fig_implied.data[0].cells.values = cur


<div class="markdown-google-sans">

### <strong>*最高最低價*</strong>
</div>

In [ ]:
def on_tick_data_high_low(tickdatahighlow: FTickDataHighLow):
    cur = list(fig_implied.data[0].cells.values)
    cur[col_implied['最高價']] = tickdatahighlow.High
    cur[col_implied['最低價']] = tickdatahighlow.Low
    with fig_implied.batch_update():
        fig_implied.data[0].cells.values = cur


<div class="markdown-google-sans">

### <strong>*結算價*</strong>
</div>

In [ ]:
def on_tick_data_settle(tickdatasettle: FTickDataSettle):
  cur = list(fig_implied.data[0].cells.values)
  cur[col_implied['目前結算價']] = tickdatasettle.CurrStl
  cur[col_implied['最新結算價']] = tickdatasettle.NewStl
  with fig_implied.batch_update():
    fig_implied.data[0].cells.values = cur

<div class="markdown-google-sans">

## <strong>*註冊*</strong>
</div>

In [ ]:
productid='CME|NQ|202412|F|' #商品代碼
api.fquote.on_tick_data_trade = on_tick_data_trade
api.fquote.on_tick_data_bid = on_tick_data_bid
api.fquote.on_tick_data_offer = on_tick_data_offer
api.fquote.on_tick_data_implied = on_tick_data_implied
api.fquote.on_tick_data_high_low = on_tick_data_high_low
api.fquote.on_tick_data_open_close = on_tick_data_open_close
api.fquote.on_tick_data_settle = on_tick_data_settle
subok, msg = api.fquote.subscribe(exchange=productid.split('|')[0], symbol=productid.split(
        '|')[1], ym=productid.split('|')[2], cp=productid.split('|')[3], strike=productid.split('|')[4])
print(f'註冊報價 {productid} {subok} {msg}')
display(fig_match)
display(fig_bid)
display(fig_off)

<div class="markdown-google-sans">

##<strong>*反註冊*</strong>
</div>

In [ ]:
#反註冊現貨價量接收
unsub_ok,msg=api.fquote.unsubscribe(exchange=productid.split('|')[0], symbol=productid.split(
        '|')[1], ym=productid.split('|')[2], cp=productid.split('|')[3], strike=productid.split('|')[4])
print(f'反註冊報價 {productid} {subok} {msg}')

<div class="markdown-google-sans">

## <strong>*查詢歷史K線行情(V1.0.0.4開始提供)*</strong>
</div>

<div class="markdown-google-sans">

### <strong>*日K*</strong>
</div>

In [ ]:
interval = "D"  # K線週期 D:日K、1K:分K
start = datetime(2025, 4, 1, 0, 0, 0)
end = datetime(2025, 6, 27, 0, 0, 0)
productkind = "1"  # 商品種類 1:期貨 2:選擇權 4:期貨價差
exchange = "CMX"  # 交易所代號
symbol = "GC"  # 商品代號
ym = "202508"  # 年月 格式: YYYYMM
cp = "F"  # 合約類型 F:期貨 C:選擇權Call P:選擇權Put
strikeprice = ""  # 履約價(選擇權用)
count = 50 # 取得K線數量
bardata_dd.drop(bardata_dd.index, inplace=True)  # 清空bardata_dd
bardatas = api.fquote.get_history_bardata(interval, start, end, productkind, exchange, symbol, ym, cp, strikeprice, count)
if (bardatas.ok):
    i = 0
    for bardata in bardatas.data:
        bardata_dd.loc[i] = [bardata.productId, bardata.productkind, bardata.date, bardata.time, bardata.open, bardata.high, bardata.low, bardata.close, bardata.volume]
        i += 1
    clear_output(wait=True)
    display(bardata_dd)
else:
    print("查詢發生錯誤: " + bardatas.error)

<div class="markdown-google-sans">

### <strong>*分K*</strong>
</div>

In [ ]:
interval = "1K"  # K線週期 D:日K、1K:分K
start = datetime(2025, 6, 27, 3, 45, 0)
end = datetime(2025, 6, 27, 5, 0, 0)
productkind = "1"  # 商品種類 1:期貨 2:選擇權 4:期貨價差
exchange = "NYM"  # 交易所代號
symbol = "CL"  # 商品代號
ym = "202509"  # 年月 格式: YYYYMM
cp = "F"  # 合約類型 F:期貨 C:選擇權Call P:選擇權Put
strikeprice = ""  # 履約價(選擇權用)
count = 50 # 取得K線數量
bardata_dd.drop(bardata_dd.index, inplace=True)  # 清空bardata_dd
bardatas = api.fquote.get_history_bardata(interval, start, end, productkind, exchange, symbol, ym, cp, strikeprice, count)
if (bardatas.ok):
    i = 0
    for bardata in bardatas.data:
        bardata_dd.loc[i] = [bardata.productId, bardata.productkind, bardata.date, bardata.time, bardata.open, bardata.high, bardata.low, bardata.close, bardata.volume]
        i += 1
    clear_output(wait=True)
    display(bardata_dd)
else:
    print("查詢發生錯誤: " + bardatas.error)

<div class="markdown-google-sans">

# <strong>*系統登出*</strong>
</div>


In [ ]:
#系統登出
api.logout()
print('登出完畢')
print(f"登入旗標{api.login_status_flag}")
